In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Cargar los datos desde los archivos TSV
train_data = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["label", "message"])
test_data = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["label", "message"])



In [ ]:
# Mostrar las primeras filas de los datos para verificar que se han cargado correctamente
print("Datos de entrenamiento:")
print(train_data.head())

In [ ]:
# Convertir las etiquetas de texto a valores numéricos
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

In [ ]:
# Tokenización y secuenciación de los mensajes
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['message'])

X_train = tokenizer.texts_to_sequences(train_data['message'])
X_test = tokenizer.texts_to_sequences(test_data['message'])

In [ ]:

max_length = 100
X_train_pad = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_length, padding='post')
X_test_pad = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_length, padding='post')

In [ ]:
# Convertir las etiquetas en arrays numpy
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [ ]:
# Crear el modelo de red neuronal
model = keras.Sequential([
    keras.layers.Embedding(5000, 16, input_length=max_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    ])

In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Entrenar el modelo
model.fit(X_train_pad, y_train, epochs=10, validation_data=(X_test_pad, y_test), verbose=2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    padded = keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_length, padding='post')
    prediction = model.predict(padded)[0][0]
    label = "spam" if prediction > 0.5 else "ham"
    return [float(prediction), label]

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
